## This notebook calculates the following metrics sourced from Cal-Adapt Tools: Wildfire
* Change in total area burned per tract per year

In [5]:
import pandas as pd
import os
import sys
import math
import geopandas as gpd
import xarray as xr
import cftime

# suppress pandas purely educational warnings
from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

sys.path.append(os.path.expanduser('../../'))
from scripts.utils.file_helpers import upload_csv_aws, filter_counties
from scripts.utils.write_metadata import append_metadata

In [7]:
ds = xr.open_dataset('MIROC5_85_AA.monthly_all.bau.mu.nc', decode_times=False)
units, reference_date = ds.time.attrs['units'].split('since')
ds['time'] = pd.date_range(start=reference_date, periods=ds.sizes['time'], freq='MS')

In [ ]:
# all of these are assuming r1i1ip1
# https://github.com/mathause/cmip_warming_levels/blob/main/warming_levels/cmip5_all_ens/csv/cmip5_warming_levels_all_ens_1850_1900_no_bounds_check.csv

# uses a 20 year average around start year
# MIROC5 4.5 -- 2063-2082
# MIROC5 8.5 -- 2039-2058
# CNRM-CM5 4.5 -- 2049-2068
# CNRM-CM5 8.5 -- 2036-2055
# HADGEM-ES 4.5 -- 2034-2053
# HADGEM-ES 8.5 -- 2026-2045
# CANESM2 4.5 -- 2022-2041
# CANESM2 8.5 -- 2017-2036

In [18]:
url = 'https://github.com/mathause/cmip_warming_levels/blob/main/warming_levels/cmip5_all_ens/csv/cmip5_warming_levels_all_ens_1850_1900.csv'
df = pd.read_csv(url, index_col=0, header=4)
df

ParserError: Error tokenizing data. C error: Expected 1 fields in line 40, saw 19


In [ ]:
def find_start_end(model, rcp):
    df = pd.read_csv(url, index_col=0)
    start_yr = df.loc[

In [14]:
def convert_start_end_year_30yr(start,end):
    start_30 = start - 5
    end_30 = end + 5
    return (start_30, end_30)

In [15]:
convert_start_end_year_30yr(2063, 2082)

(2058, 2087)

In [12]:
2086-2058

28

In [13]:
2082-2063

19

In [8]:
ds

<xarray.Dataset>
Dimensions:   (lon: 173, lat: 179, time: 1764)
Coordinates:
  * lon       (lon) float64 -124.5 -124.5 -124.4 -124.3 ... -113.9 -113.8 -113.8
  * lat       (lat) float64 31.59 31.66 31.72 31.78 ... 42.53 42.59 42.66 42.72
  * time      (time) datetime64[ns] 1953-01-01 1953-02-01 ... 2099-12-01
Data variables:
    hectares  (time, lat, lon) float32 ...

ValueError: Dataset.plot cannot be called directly. Use an explicit plot method, e.g. ds.plot.scatter(...)